In [2]:
# !pip install -r ../requirements.txt

# load model

In [4]:
import torch
import torchaudio
from transformers import AutoProcessor, SeamlessM4Tv2ForSpeechToText

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2ForSpeechToText.from_pretrained("facebook/seamless-m4t-v2-large").to(device)

model.device

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

device(type='cuda', index=0)

# load dataset

In [6]:
from datasets import load_dataset

In [7]:
dataset_name = "hsekhalilian/commonvoice"
dataset = load_dataset(dataset_name, split="dev")
dataset = dataset.select(range(1000))

# text to speech

In [3]:
from IPython.display import Audio

# from text
text_inputs = processor(text = "Hello, my dog is cute, how are you?", src_lang="eng", return_tensors="pt")
audio_array_from_text = model.generate(**text_inputs, tgt_lang="pes")[0].cpu().numpy().squeeze()


sample_rate = model.config.sampling_rate
Audio(audio_array_from_text, rate=sample_rate)

[W712 14:35:52.048399654 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


In [4]:
text_inputs = processor(text = "سلام حالتون چطوره من حسین هستم و تهران زندگی میکنم شما چطور؟", src_lang="pes", return_tensors="pt")
audio_array_from_text = model.generate(**text_inputs, tgt_lang="pes")[0].cpu().numpy().squeeze()


sample_rate = model.config.sampling_rate
Audio(audio_array_from_text, rate=sample_rate)

[W712 14:35:56.476669449 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


# speech to text

In [8]:
TARGET_SAMPLING_RATE = 16000

In [9]:
sample = dataset[0]

audio = sample["audio"]
speech_array, sampling_rate = audio["array"], audio["sampling_rate"]
audio_inputs = processor(audios=speech_array, return_tensors="pt", sampling_rate=TARGET_SAMPLING_RATE).to(device)

output_tokens = model.generate(**audio_inputs, tgt_lang="pes")
transcription = processor.decode(output_tokens[0].tolist(), skip_special_tokens=True)

print("🔊 Ground Truth:", sample["sentence"])
print("📝 Transcription:", transcription)
print("-" * 50)

🔊 Ground Truth: این اولین قدم برای تغییر خودم
📝 Transcription: این اولین قدم برای تغییر خودم
--------------------------------------------------


In [25]:
from torch.utils.data import DataLoader

batch_size = 100  # Set your desired batch size
samples = dataset.select(range(1000))  # Select your subset

# Collate function for DataLoader
def collate_fn(batch):
    speech_arrays = [sample["audio"]["array"] for sample in batch]
    sampling_rates = [sample["audio"]["sampling_rate"] for sample in batch]
    sentences = [sample["sentence"] for sample in batch]
    return speech_arrays, sampling_rates, sentences

# Create DataLoader
loader = DataLoader(samples, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

for speech_arrays, sampling_rates, sentences in loader:
    # Assuming all audios have the same sampling rate
    audio_inputs = processor(
        audios=speech_arrays,
        return_tensors="pt",
        sampling_rate=TARGET_SAMPLING_RATE,
        padding=True
    ).to(device)

    output_tokens = model.generate(**audio_inputs, tgt_lang="pes")

    transcriptions = processor.batch_decode(output_tokens, skip_special_tokens=True)

    for sentence, transcription in zip(sentences, transcriptions):
        print("🔊 Ground Truth:", sentence)
        print("📝 Transcription:", transcription)
        print("-" * 50)


🔊 Ground Truth: این اولین قدم برای تغییر خودم
📝 Transcription: این اولین قدم برای تغییر خودم
--------------------------------------------------
🔊 Ground Truth: با خنده ای ترسناک چرا وحشت کردین؟ چرا تهمت می زنی؟
📝 Transcription: با خنده ی ترسناک چرا وحشت کردید؟ چرا تهمت می زنید؟
--------------------------------------------------
🔊 Ground Truth: من همه جا دنبالت گشتم
📝 Transcription: من و همه جا دنبالت گشتم
--------------------------------------------------
🔊 Ground Truth: افسانهها میگن سگها واسطهی دنیای زندهها با مردههان
📝 Transcription: افسانهها میگن تکا وسط دنیای زندهها و مردهها
--------------------------------------------------
🔊 Ground Truth: فكر می کنم همین جا باید تمومش کنیم
📝 Transcription: فکر میکنم همینجا باید تمومش کنیم
--------------------------------------------------
🔊 Ground Truth: اَفراسیاب
📝 Transcription: افراسیاب
--------------------------------------------------
🔊 Ground Truth: طاهره چی بهش گفتی رنگش پرید
📝 Transcription: تا اره چی بهش گفتی رنگ اسپرید؟
---------------